In [ ]:
from typing import TypedDict, List, Dict, Any, Optional
from langgraph.graph import add_messages
from IPython.display import Image, display
class FormState(TypedDict):
    messages: List[str]  # conversation log
    form_fields: List[Dict[str, Any]]  # schema/payload from frontend
    filled_fields: Dict[str, Any]  # field_id → value
    pending_field: Optional[str]  # field_id currently being asked

In [2]:
def parse_input(state: FormState) -> FormState:
    msg = state["messages"][-1]
    # Simple heuristic or use LLM extraction here
    extracted = {}  
    for field in state["form_fields"]:
        if field["field_label"].lower() in msg.lower():
            extracted[field["data_id"]] = "some parsed value"
    state["filled_fields"].update(extracted)
    return state

In [3]:
def fill_known_fields(state: FormState) -> FormState:
    missing = [
        f["data_id"] for f in state["form_fields"]
        if f["is_required"] and f["data_id"] not in state["filled_fields"]
    ]
    if missing:
        state["pending_field"] = missing[0]
    else:
        state["pending_field"] = None
    return state

In [4]:
def ask_missing_field(state: FormState) -> str:
    if state["pending_field"]:
        field = next(f for f in state["form_fields"] if f["data_id"] == state["pending_field"])
        return f"Can you please provide: {field['field_label']}?"
    return "All fields completed!"

In [5]:
def collect_answer(state: FormState, user_input: str) -> FormState:
    if state["pending_field"]:
        state["filled_fields"][state["pending_field"]] = user_input
    state["pending_field"] = None
    return state

In [15]:
import os
from typing import Literal, Optional
from langgraph.graph import StateGraph, MessagesState, START, END
from IPython.display import Image, display
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from pprint import pprint
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
workflow = StateGraph(FormState)

workflow.add_node("parse_input", parse_input)
workflow.add_node("fill_known_fields", fill_known_fields)
workflow.add_node("ask_missing_field", ask_missing_field)
workflow.add_node("collect_answer", collect_answer)

workflow.add_edge(START, "parse_input")
workflow.add_edge("parse_input", "fill_known_fields")
workflow.add_edge("fill_known_fields", "ask_missing_field")
workflow.add_edge("ask_missing_field", "collect_answer")
workflow.add_edge("collect_answer", "fill_known_fields")
workflow.add_edge("fill_known_fields", END)

app = workflow.compile()

from IPython.display import display, clear_output, HTML
from pprint import pprint

# --- Conversation Display Helper ---
def display_conversation(messages, filled_fields=None):
    clear_output(wait=True)  # clear previous output to simulate live chat
    html = "<div style='font-family: Arial; line-height:1.5;'>"
    
    for msg in messages:
        if msg.startswith("User:"):
            html += f"<div style='margin:6px 0;'><b style='color:blue;'>{msg}</b></div>"
        elif msg.startswith("Agent:"):
            html += f"<div style='margin:6px 0;'><b style='color:green;'>{msg}</b></div>"
    
    if filled_fields:
        html += "<hr><b>✅ Form Completed:</b><pre>" + str(filled_fields) + "</pre>"
    
    html += "</div>"
    display(HTML(html))

# --- Execution Loop with Notebook Display ---
state = {
    "messages": ["User: Can you help me fill this form? My name is John Doe."],
    "form_fields": [
        {"data_id": "name", "field_label": "Full Name", "is_required": True},
        {"data_id": "email", "field_label": "Email Address", "is_required": True},
        {"data_id": "age", "field_label": "Age", "is_required": False}
    ],
    "filled_fields": {},
    "pending_field": None
}

while True:
    # Step workflow forward
    state = app.step(state)

    if not state["pending_field"]:  # Completed form
        state["messages"].append("Agent: Great! The form is complete.")
        display_conversation(state["messages"], state["filled_fields"])
        break
    else:
        # Agent asks next missing field
        question = ask_missing_field(state)
        state["messages"].append(f"Agent: {question}")
        display_conversation(state["messages"])
        
        # Simulate user reply (replace input() with front-end connector)
        user_input = input("User: ")
        state["messages"].append(f"User: {user_input}")
        state = collect_answer(state, user_input)



AttributeError: 'CompiledStateGraph' object has no attribute 'step'

AttributeError: 'CompiledStateGraph' object has no attribute 'step'